In [15]:
import tensorflow as tf
# მივიღოთ GPU-ს სახელი
device_name = tf.test.gpu_device_name()
# დავაფორმატოთ output-ი
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [16]:
import torch
# თუ GPU ნაპოვნია
if torch.cuda.is_available():    
    # ვუბრძანოთ pytorch-ს რომ გამოიყენოს იგი    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [17]:
!pip install transformers

In [18]:
import pandas as pd
# ჩავტვირთოთ მონაცემები pandas-ის dataframe-ში.
df = pd.read_csv("./data.txt", delimiter=',', header=None, names=['sentence', 'label'])
# დავბეჭდოთ დასასწავლი წინადადებების რაოდენობა
print('Number of training sentences: {:,}\n'.format(df.shape[0]))
# ვნახოთ რამდენიმე დასასწავლი წინადადების მაგალითი
df.sample(2)

Number of training sentences: 2



,sentence,label
0,ბალანსი მაჩვენე ჯიგარო,0
1,რა რიცხვია დღეს?,1


In [19]:
# ცალ-ცალკე მასივებში ჩავსვათ წინადადებები და მათი კლასები
sentences = df.sentence.values
labels = df.label.values
print(sentences)
print(labels)

['ბალანსი მაჩვენე ჯიგარო' 'რა რიცხვია დღეს?']
[0 1]


In [20]:
from transformers import BertTokenizer
# ჩამოვტვირთოთ BERT-ის ტოკენიზატორი.
print('Loading BERT tokenizer...')
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Loading BERT tokenizer...


In [21]:
# მოვახდინოთ ყველა წინადადების ტოკენიზაცია და ავსახოთ მათი ტოკენი სიტყვის ID-ში
input_ids = []

for sent in sentences:
    # `encode` ფუნქცია იზამს შემდეგს:
    #   (1) მოახდენს წინადადების ტოკენიზაციას
    #   (2) დასაწყისში  `[CLS]` ტოკენს ჩასვამს
    #   (3) ბოლოში მიადგამს `[SEP]` ტოკენს.
    #   (4) ასახავს ტოკენს თავის ID-ში
    encoded_sent = tokenizer.encode(
                        sent,                      # დასამუშავებელი წინადადება
                        add_special_tokens = True, # ჩაამატე '[CLS]' და '[SEP]'
                   )
    
    # დამუშავებული წინადადებები ჩავამატოთ სიაში
    input_ids.append(encoded_sent)

# დავბეჭდოთ მენულე წინადადება ახლა უკვე როგორც ID-ების სია.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  ბალანსი მაჩვენე ჯიგარო
Token IDs: [101, 1565, 11577, 25559, 94934, 13266, 1575, 11577, 111518, 88154, 17076, 1595, 13266, 96847, 28068, 102]


In [22]:
sent = "ბალანსი მაჩვენე ჯიგარო"

# ინგლისური სიტყვების ტოკენიზაცია მუშაობს
print(tokenizer.convert_tokens_to_ids(["car"]))

# დავბეჭდოთ წინადადება sent-ის ტოკენიზაციის შედეგი
print(tokenizer.tokenize(sent))

# ასო 'ბ'-ს ID არის 1565
print(tokenizer.convert_tokens_to_ids(["ბ"]))

[13000]
['ბ', '##ა', '##ლა', '##ნს', '##ი', 'მ', '##ა', '##ჩ', '##ვენ', '##ე', 'ჯ', '##ი', '##გა', '##რო']
[1565]


In [23]:
# ID-ების სიების მაქსიმალური სიგრძე
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  16


In [24]:
# ვიხმაროთ keras-ის დამხმარე ფუნქცია `pad_sequences` რომ ყველა სიას იგივე ზომა მივცეთ
from keras.preprocessing.sequence import pad_sequences

# დავაწესოთ წინადადების მაქსიმალური სიგრძე (სიტყვაზე 120)
MAX_LEN = 120

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))
# [PAD] ტოკენის ID არის 0, ამიტომ შევავსოთ ყველა სია 120 სიგრძემდე 0-ებით
# "post" ანიშნებს, რომ [PAD] ტოკენები გვინდა ჩავსვათ სიის ბოლოში და არა თავში
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
print('\Done.\n\n')

print("Example: \n\n\n",input_ids[0])


Padding/truncating all sentences to 120 values...

Padding token: "[PAD]", ID: 0
\Done.


Example: 


 [   101   1565  11577  25559  94934  13266   1575  11577 111518  88154
  17076   1595  13266  96847  28068    102      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0]


In [25]:
# შევქმნათ სია, რომელშიც გვექნება ყველა წინადადების ყურადღების mask-ები
attention_masks = []

for sent in input_ids:
    
    # შევქმნათ ყურადღების mask-ი:
    #   თუ ტოკენის ID არის 0, მაშინ [PAD] ტოკენი ყოფილა და mask არის 0
    #   თუ ტოკენის ID არის  > 0, მაშინ არაა [PAD] ტოკენი და mask არის 1
    att_mask = [int(token_id > 0) for token_id in sent]
    
    attention_masks.append(att_mask)

In [26]:
# ვიხმაროთ train_test_split რომ დავყოთ წინადადებები დასასწავლ და გასატესტ ნაწილებად
from sklearn.model_selection import train_test_split
# ვიხმაროთ 90% სწავლებისთვის და 10% ტესტირებისთვის
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2021, test_size=0.1)
# იგივე ვქნათ mask-ებისთვის
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2021, test_size=0.1)

In [27]:
# ყველაფერი ვაქციოთ pytorch-ის ტენზორებად (სწავლებისთვის საჭიროა)
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [28]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# DataLoader-ს წინასწარ უნდა ვუთხრათ batch-ების ზომა რომელსაც სწავლებისას ვიხმართ
# რადგან ჩვენ გადასწავლებას ვახდენთ, პატარა რიცხვიც კმარა (როგორიცაა 32)
batch_size = 32

# შევქმნათ DataLoader-ი დასასწავლი წინადადებებისთვის
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# შევქმნათ DataLoader-ი გასატესტი წინადადებებისთვის
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [29]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
# ჩამოვტვირთოთ BertForSequenceClassification, წინასწარ ნასწავლი BERT-ის მოდელი
# რომელსაც თავზე აქვს მიდგმული კლასიფიქატორი ქსელი

model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels = 2, # კლასების რაოდენობა - ჯერჯერობით 2 კლასი გვაქვს   
    output_attentions = False, # არ დავაბრუნოთ attention weight-ები
    output_hidden_states = False, # არ დავაბრუნოთ ფარული მეხსიერება
)
# ვიხმაროთ GPU გამოთვლებისთვის
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [30]:
# დავბეჭდოთ ბერტის პარამეტრები
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (119547, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              

In [31]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate
                  eps = 1e-8 # args.adam_epsilon
                )
from transformers import get_linear_schedule_with_warmup

# ვთ. გვაქვს 4 ეპოქა. თითო ეპოქაში დასასწავლ მონაცემებიდან ყველას
# ერთხელ მაინც ექნება საშუალება რომ გავლენა იქონიოს ქსელის პარამეტრების ცვლილებაზე
epochs = 4

# total_steps = batches * epochs, სადაც batch არის დასასწავლ მონაცემთა რაღაც ქვესიმრავლე
# რომელზეც ეპოქის დროს ისწავლის ქსელი
total_steps = len(train_dataloader) * epochs

# scheduler განსაზღვრავს სწავლების თანმიმდევრობას
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)